# Scraping

In [10]:
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

## 1. Selenium
## 1.1 Scrape links

In [48]:
url = "https://ec.europa.eu/commission/presscorner/home/en?dotyp=&keywords=GHG&commissioner="
driver = webdriver.Chrome()
driver.get(url)

In [52]:
page_content = driver.find_element(By.ID, 'news-block')
# Now, find all <a> tags *within* that specific 'page_content' element
all_links= page_content.find_elements(By.TAG_NAME, 'a')

links = []
for link in all_links:
    href = link.get_attribute('href')
    text = link.text
    #print(f"Link Text: '{text}', URL: '{href}'")
    links.append(href)
links = [link for link in links if link!=None and len(link)>0]
links[:2]

['https://ec.europa.eu/commission/presscorner/detail/en/mex_25_1333',
 'https://ec.europa.eu/commission/presscorner/detail/en/mex_25_1287']

## 1.2 Scrape all links

In [87]:
#driver = webdriver.Chrome()
def scrape_page(url):
    driver.get(url)
    time.sleep(4)
    #page_title
    page_title = driver.find_element(By.TAG_NAME, 'h1').text
    #page_content
    page_content = driver.find_element(By.CLASS_NAME, 'ecl-col-l-9').text
    #date
    els = driver.find_elements(By.CLASS_NAME, 'ecl-page-header__meta-item')
    els = [el.text for el in els]
    date = next((item for item in els if re.search(r'\d{4}', item)), None)
    
    final_text = f"{page_title}\n{page_content}"
    return date.replace(",","").replace(" ", "_"), final_text

### 1.2.1 Example

In [63]:
date, final_text = scrape_page(links[1])
final_text[:400]

"Daily News 20 / 05 / 2025\nCommission welcomes political agreement on the progressive rollout of Europe's New Digital Border System\nThe European Commission welcomes yesterday's provisional political agreement by the European Parliament and the Council on the Commission's proposal for a progressive rollout of Europe's new digital border system, the Entry/Exit System (EES).\nThis agreement will allow "

### 1.2.2 Scrape all

In [65]:
scrape_dictionary = {}
from tqdm import tqdm
for link in tqdm(links):
    try:
        date, final_text = scrape_page(link)
        scrape_dictionary[date] = final_text
    except:
        print("LINK DIDNT WORK", link)


### 1.2.3 Convert results to data frame

In [82]:
import pandas as pd
df_text = pd.DataFrame.from_dict(scrape_dictionary, orient = 'index', columns = ['text'])
df_text.index.name = 'date'
df_text.tail(2)

,text
date,
Apr_12_2024,Questions and Answers on the revised Energy Pe...
Apr_5_2024,Commission approves €350 million German State ...


### 1.2.4 Export

In [83]:
import os
export_path = "../../data/examples/module_4"
os.listdir(export_path)

[]

In [85]:
file_name = f"{export_path}/eu_press_releases_ghg.csv"
df_text.to_csv(file_name)

In [88]:
pd.read_csv(file_name).head(2)

,date,text
0,May_26_2025,Daily News 26 / 05 / 2025\nEU organises firefi...
1,May_20_2025,Commission approves €1.2 billion Dutch State a...
